# Wordle Solver

In [1]:
from wordle.utils import Wordle

%load_ext autoreload
%autoreload 2

### Play Wordle

In [2]:
# Initialize Wordle class
wordle = Wordle()

# Main function that returns a list of possible words
green_letters = "ai"
green_positions = [1, 4]
yellow_letters = "ia"
yellow_positions = [2, 3]
gray_letters = "trelon"

# Find words in a list of possible words
words = wordle.find_words(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    gray_letters,
    answer_word_list=True,
)

# Choose a word to play
wordle.choose_word_to_play(word_threshold=2)

Number of possible words: 2
aphid 7.0
affix 6.0


In [3]:
# Repetitive letters
letters_df = wordle.repetitive_letters()
print(letters_df.to_string(index=False))

Letters  Count
      A      2
      F      2
      I      2
      X      1
      P      1
      H      1
      D      1


In [4]:
df = wordle.load_data()
df

,Names,Games_Won
0,Murilo,328
1,Barbara,142
2,Draw,323


In [5]:
# Update score
score_df = wordle.score()
score_df

,Names,Games_Won
0,Murilo,329
1,Barbara,142
2,Draw,323


In [6]:
# Print results
print(score_df.to_string(index=False))

  Names  Games_Won
 Murilo        329
Barbara        142
   Draw        323


#### Git commit message

In [7]:
# git commit message
commit_message = wordle.get_commit_message()
print(commit_message)

git commit -m "updated wordle 329-142-323"


#### Set custom score

In [8]:
# help(wordle.reset_score)

In [9]:
# help(wordle.set_score)

In [10]:
# # Set custom score
# df = wordle.set_score(m_score=311, b_score=132, draw_score=308)
# df

#### Reset score

In [11]:
## Reset score
# wordle.reset_score()